In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib

Using matplotlib backend: MacOSX


In [2]:
def test_convergence(list_losses, eps):
    if len(list_losses)<3:
        return False
    else:
        criterion = nn.MSELoss()
        crit1 = criterion(torch.tensor(list_losses[-3]), torch.tensor(list_losses[-2]))
        crit2 = criterion(torch.tensor(list_losses[-1]), torch.tensor(list_losses[-2]))
        if crit1 + crit2 < eps:
            return True
        return False

In [3]:
def fun(x):
    x_bar = torch.mean(x)
    N = x.shape[0]
    one = torch.ones(N)
    return torch.exp(x_bar + x_bar**3)*(one + 3*x**2)

In [4]:
class DeepDerSet(nn.Module):

    def __init__(self, N):
        super().__init__() # Runs initialisation of nn.Module
        self.fc1 = nn.Linear(1, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 128)
        self.fc5 = nn.Linear(128, 1024)
        
        self.psi1 = nn.Linear(N + 1024, 512)
        self.psi2 = nn.Linear(512, 256)
        self.psi3 = nn.Linear(256, N)
        
        self.N = N

    def phi(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

    def psi(self, x, y):
        
        z = torch.cat([x, y])
        z = F.relu(self.psi1(z))
        z = F.relu(self.psi2(z))
        z = self.psi3(z)
        return z
        
    def forward(self, x):
        N = self.N
        y = torch.zeros(N, 1024)
        for i in range(N):
            y[i] = self.phi(x[i].view(-1))
        y = y.max(0).values
        return self.psi(x, y)

In [5]:
N = 10
n_data = 100
x = torch.normal(torch.zeros(n_data, N), torch.ones(n_data, N)) 
synth = torch.zeros(n_data, N)
for i in range(0,n_data):
    f = fun(x[i, :])
    synth[i,:] = fun(x[i, :])

In [6]:
synth.shape

torch.Size([100, 10])

In [7]:
batches = 10
net = DeepDerSet(N).float()
optimizer = optim.Adam(net.parameters(), lr=0.01) # Corresponds to evything that is adjustable
criterion = nn.MSELoss()
EPOCHS = 100
outputs_epoch = []

losses_epoch = []
for epoch in range(EPOCHS):
    loss_batch = []
    for i in range(batches):
        net.zero_grad()
        k = int(n_data /batches)
        dataX = x[i:i+k,:]
        dataY = synth[i:i+k,:]
        tot_loss = 0
        net.zero_grad()
        for j , (X, Y) in enumerate(zip(dataX, dataY)):
            output = net(X)
            loss = criterion(output, Y)
            tot_loss += loss
            loss_batch.append(loss)
        tot_loss.backward() # Backpropagate the loss
        optimizer.step() # Adjusts the steps
    losses_epoch.append(loss_batch)
    print('LOSS = ', tot_loss)
    if test_convergence(losses_epoch, 0.1):
        break

LOSS =  tensor(209.7650, grad_fn=<AddBackward0>)
LOSS =  tensor(287.7296, grad_fn=<AddBackward0>)
LOSS =  tensor(145.0028, grad_fn=<AddBackward0>)
LOSS =  tensor(98.6237, grad_fn=<AddBackward0>)
LOSS =  tensor(88.2992, grad_fn=<AddBackward0>)
LOSS =  tensor(81.5720, grad_fn=<AddBackward0>)
LOSS =  tensor(65.5426, grad_fn=<AddBackward0>)
LOSS =  tensor(54.4400, grad_fn=<AddBackward0>)
LOSS =  tensor(49.4401, grad_fn=<AddBackward0>)
LOSS =  tensor(38.0237, grad_fn=<AddBackward0>)
LOSS =  tensor(29.9978, grad_fn=<AddBackward0>)
LOSS =  tensor(22.6054, grad_fn=<AddBackward0>)
LOSS =  tensor(18.2709, grad_fn=<AddBackward0>)
LOSS =  tensor(15.2249, grad_fn=<AddBackward0>)
LOSS =  tensor(12.8097, grad_fn=<AddBackward0>)
LOSS =  tensor(10.4496, grad_fn=<AddBackward0>)
LOSS =  tensor(8.5627, grad_fn=<AddBackward0>)
LOSS =  tensor(7.1972, grad_fn=<AddBackward0>)
LOSS =  tensor(6.2132, grad_fn=<AddBackward0>)
LOSS =  tensor(5.7475, grad_fn=<AddBackward0>)


In [11]:
len(losses_epoch)

20

In [12]:
k = 5

In [20]:
plt.close()
for i, tot_losses in enumerate(losses_epoch[4:]):
    if i%k == 0:
        plt.plot(tot_losses, label=str(i+4))
plt.legend()
plt.title("DeepDerSet")

Text(0.5, 1.0, 'DeepDerSet')

In [62]:
output

tensor([1.0580, 2.0044, 4.6719, 3.1931, 2.8831, 6.4153, 1.0592, 2.9822, 5.4159,
        3.1815], grad_fn=<AddBackward0>)

In [63]:
list(net.parameters())

[Parameter containing:
 tensor([[-0.3056],
         [ 0.7570],
         [ 0.9986],
         [ 0.7285],
         [-0.7082],
         [ 0.9352],
         [-0.9268],
         [-0.2776],
         [ 0.7805],
         [ 0.7518],
         [ 0.9174],
         [ 0.3512],
         [-0.1418],
         [ 0.5707],
         [-0.6647],
         [-0.6440],
         [-0.9561],
         [-0.0090],
         [-0.0794],
         [ 0.5561],
         [ 0.8119],
         [ 0.7834],
         [ 0.3158],
         [-0.3996],
         [ 0.9475],
         [-0.3849],
         [-0.4243],
         [-0.3241],
         [ 0.1463],
         [-0.8588],
         [-0.9332],
         [-0.2000],
         [ 0.1989],
         [-0.8560],
         [-0.3007],
         [-0.7218],
         [-0.4654],
         [ 0.3900],
         [-0.0189],
         [-0.1893],
         [-0.1125],
         [ 0.8380],
         [ 0.1887],
         [-0.3842],
         [-0.2653],
         [-0.5734],
         [ 0.5967],
         [-0.3252],
         [ 0.8905